In [1]:
# Importación librerías
import pandas as pd
import numpy as np

In [2]:
# Lectura de datos
df = pd.read_excel("reporte_diario_campaña_limpio.xlsx").drop(columns='Unnamed: 0')


In [3]:
def promedio_movil_val(dfs,target, linea):
    datos = dfs[dfs["linea"] == linea]
    n_periodos=datos[datos['fecha'].dt.year == 2024].shape[0]
    print(n_periodos)
    datos = datos[['dia_semana',target]].dropna()
    datos_entrenamiento = datos[:-n_periodos]
    datos_prueba = datos[-n_periodos:]

    reporte_diasemana_linea=datos_entrenamiento[[target,"dia_semana"]]
    
    datos_=datos_prueba[["dia_semana"]][:n_periodos].values
    
    new_pre=[]
    dia_semana=[]
    for i in datos_:

        temp=reporte_diasemana_linea[reporte_diasemana_linea["dia_semana"]==i[0]][-7:][target]
        if (len(temp.values)==7):
            w=np.array([1,6,11,14,17,21,30])
        elif (len(temp.values)==6):
            w=np.array([7,11,14,17,21,30])
        elif (len(temp.values)==5):
            w=np.array([5,15,20,25,35])
        elif (len(temp.values)==4):
            w=np.array([16,21,26,37])
        elif (len(temp.values)==3):
            w=np.array([25,30,45])
        elif (len(temp.values)==2):
            w=np.array([30,70])
        elif (len(temp.values)==1):
            w=np.array([100])
        new_pre.append(sum(temp.values*w/100))
        dia_semana.append(i)

    reporte_diasemana_linea=pd.concat([reporte_diasemana_linea,pd.DataFrame({"dia_semana":dia_semana,target:new_pre})])

    return(reporte_diasemana_linea[target][-n_periodos:].values)


In [4]:
validacion_calls = []
lineas = df["linea"].unique()
for linea in lineas:
    print(f"Procesando línea: {linea}")
    val_wfm = promedio_movil_val(df, "interpolado_real_calls", linea)
    
    # Crear un DataFrame de la longitud específica de val_wfm
    val_promedio_movil = pd.DataFrame(index=range(len(val_wfm)))
    val_promedio_movil["val_wfm"] = val_wfm
    val_promedio_movil["linea"] = linea
    
    validacion_calls.append(val_promedio_movil)

Procesando línea: linea_1
98
Procesando línea: linea_2
98
Procesando línea: linea_3
98
Procesando línea: linea_4
98
Procesando línea: linea_5
96


In [5]:
validacion_wfm_calls = pd.concat(validacion_calls, ignore_index=True)
val_calls = pd.read_excel("validacion_diaria_calls.xlsx").drop(columns='Unnamed: 0')
val_calls["pred_wfm"] = validacion_wfm_calls["val_wfm"]

In [6]:
val_calls.to_excel("validacion_diaria_calls_wfm.xlsx")


In [4]:
validacion_aht = []
lineas = df["linea"].unique()
for linea in lineas:
    print(f"Procesando línea: {linea}")
    val_wfm = promedio_movil_val(df, "interpolado_real_aht", linea)
    
    # Crear un DataFrame de la longitud específica de val_wfm
    val_promedio_movil = pd.DataFrame(index=range(len(val_wfm)))
    val_promedio_movil["val_wfm"] = val_wfm
    val_promedio_movil["linea"] = linea
    
    validacion_aht.append(val_promedio_movil)

Procesando línea: linea_1
98
Procesando línea: linea_2
98
Procesando línea: linea_3
98
Procesando línea: linea_4
98
Procesando línea: linea_5
96


In [6]:
validacion_wfm_sht = pd.concat(validacion_aht, ignore_index=True)
val_aht = pd.read_excel("validacion_diaria_aht.xlsx").drop(columns='Unnamed: 0')
val_aht["pred_wfm"] = validacion_wfm_sht["val_wfm"]

In [7]:
val_aht.to_excel("validacion_diaria_aht_wfm.xlsx")
